Submitting various things for end of grant.

In [1]:
import pandas
import gcat
import paramiko
import json
from IPython import display


In [2]:
from curation_common import *

In [3]:
from htsworkflow.submission.encoded import Document
from htsworkflow.submission.aws_submission import run_aws_cp

In [4]:
server = ENCODED('www.encodeproject.org')
server.load_netrc()

In [5]:
spreadsheet_name = 'ENCODE Submission for Y4Q5'

# Submit Documents

Need to send ATAC Protocol

In [6]:
atac_uuid = '0fc44318-b802-474e-8199-f3b6d708eb6f'
atac = Document(os.path.expanduser('~/proj/encode3-curation/Wold_Lab_ATAC_Seq_protocol_December_2016.pdf'),
                'general protocol',
                'ATAC-Seq experiment protocol for Wold lab',
                )
body = atac.create_if_needed(server, atac_uuid)
print(body['@id'])

/documents/0fc44318-b802-474e-8199-f3b6d708eb6f/


# Submit Annotations

In [7]:
sheet = gcat.get_file(spreadsheet_name, fmt='pandas_excel')
annotations = sheet.parse('Annotations', header=0)
created = server.post_sheet('/annotations/', annotations, verbose=True, dry_run=True)
print(len(created))

0


In [8]:
if created:
    annotations.to_excel('/tmp/annotations.xlsx', index=False)

# Submit files

In [9]:
sheet = gcat.get_file(spreadsheet_name, fmt='pandas_excel')
files = sheet.parse('Files', header=0)
created = server.post_sheet('/files/', files, verbose=True, dry_run=True)
print(len(created))

16


# Fix messed up files

In [28]:
def fix_files(files, dry_run=True):
    uploads = []
    for i, row in files.iterrows():
        accession = row['accession']
        uri = '/files/{}/'.format(accession)
        obj = server.get_json(uri)
        updates = {}
        if obj['md5sum'] != row['md5sum']:
            print('need to update md5sum', obj['md5sum'])
            updates['md5sum'] = row['md5sum']
        if obj['submitted_file_name'] != row['submitted_file_name']:
            print('need to update filename', obj['submitted_file_name'], row['submitted_file_name'])
            updates['submitted_file_name'] = row['submitted_file_name']
        if 'file_size' in obj:
            if obj['file_size'] != row['file_size:skip']:
                print('need to update file_size', obj['file_size'], row['file_size:skip'])
                updates['file_size'] = row['file_size:skip']
        else:
            print(obj['accession'], 'does not have a file_size')
            updates['file_size'] = row['file_size:skip']

        if len(updates) > 0:
            print('updating {} {}'.format(uri, updates))
            if not dry_run:
                display.display_pretty(server.patch_json(uri, updates))

        #upload = server.get_json('/files/{}/upload'.format(row['accession']))
        #uploads.append(upload)
        #creds = upload['@graph'][0]['upload_credentials']

        #if 'myoblast' in row['submitted_file_name']:
        #    print(row['submitted_file_name'],creds)
        #    run_aws_cp(row['submitted_file_name'], creds)
fix_files(files, dry_run=True)

In [25]:
from IPython import display
display.display_pretty({'a':1})

{'a': 1}

In [29]:
def review_errors(files):
    for i, row in files.iterrows():
        accession = row['accession']
        uri = '/files/{}/'.format(accession)
        obj = server.get_json(uri)

        if 'content_error_detail' in obj:
            print(accession, obj['content_error_detail'])
            print('    ', obj['submitted_file_name'])

review_errors(files)

ENCFF382URM Fastq metadata-specified file size 118584 doesn’t match the calculated file size 119013
     10T_half_biorep1.bigBed
ENCFF136WLJ Fastq metadata-specified file size 111873 doesn’t match the calculated file size 112101
     10T_half_biorep2.bigBed
ENCFF230XUM Fastq metadata-specified file size 118915 doesn’t match the calculated file size 119343
     10T_half_mock_diff_biorep1.bigBed
ENCFF378ULY Fastq metadata-specified file size 111634 doesn’t match the calculated file size 111884
     10T_half_mock_diff_biorep2.bigBed
ENCFF321IHA Fastq file metadata-specified md5sum 19b8e6b76cba2a782db6025430a240da  does not match the calculated md5sum 19b8e6b76cba2a782db6025430a240da
     c2c12_myoblast_biorep1.bed.gz
ENCFF556RDS Fastq metadata-specified file size 119900 doesn’t match the calculated file size 120266
     c2c12_myoblast_biorep1.bigBed
ENCFF451IMU Fastq metadata-specified file size 112542 doesn’t match the calculated file size 112822
     c2c12_myoblast_biorep2.bigBed


In [20]:
'19b8e6b76cba2a782db6025430a240da' == '19b8e6b76cba2a782db6025430a240da'

True

In [13]:
%debug

> <ipython-input-12-f58610c5bee2>(14)fix_files()
     12             print('need to update filename', obj['submitted_file_name'], row['submitted_file_name'])
     13             updates['submitted_file_name'] = row['submitted_file_name']
---> 14         if obj['file_size'] != row['file_size:skip']:
     15             updates['file_size'] = row['file_size:skip']
     16 

ipdb> p obj
{'lab': {'schema_version': '3', '@type': ['Lab', 'Item'], 'country': 'USA', 'fax': '', 'title': 'Barbara Wold, Caltech', 'address2': '1200 California Blvd; MC156-29', 'pi': '/users/0598c868-0b4a-4c5b-9112-8f85c5de5374/', 'postal_code': '91125', 'awards': ['/awards/U54HG006998/', '/awards/UM1HG009443/'], 'name': 'barbara-wold', 'city': 'Pasadena', 'institute_label': 'Caltech', 'state': 'CA', 'status': 'current', 'phone1': '626-395-4916', '@id': '/labs/barbara-wold/', 'phone2': '626-395-4923', 'institute_name': 'California Institute of Technology', 'address1': 'Department of Biochemistry and Molecular Biophy

In [11]:
#server.get_json('/files/{}/upload'.format(row['accession']))

NameError: name 'row' is not defined

In [ ]:
obj['@graph']

In [ ]:
from htsworkflow.submission.aws_submission import run_aws_cp

In [ ]:
for row in created:
    #print(row['submitted_file_name'], row['upload_credentials'])
    if not os.path.exists(row['submitted_file_name']):
        print('missing', row['submitted_file_name'])
    else:
        run_aws_cp(row['submitted_file_name'], row['upload_credentials'])

In [ ]:
if created:
    files.to_excel('/tmp/files.xlsx', index=False)

In [ ]:
for row in created:
    print(row['accession'])

# Update Annotations With biorep comment

In [ ]:
server.patch_json?

# Register Biosamples

In [ ]:
#y4q1 = gcat.get_file(spreadsheet_name, fmt='pandas_excel')
#biosample = y4q1.parse('Biosamples', header=0)
#created = server.post_sheet('/biosamples/', biosample, verbose=True, dry_run=True)
#print(len(created))

In [ ]:
#biosample.to_excel('/dev/shm/biosamples.xlsx', index=False)

# Register Experiments

In [ ]:
y4q1 = gcat.get_file(spreadsheet_name, fmt='pandas_excel')
experiments = y4q1.parse('Experiments', header=0)

created = server.post_sheet('/experiments/', experiments, verbose=True, dry_run=True)

In [ ]:
len(created)

In [ ]:
if created:
    experiments.to_excel('/tmp/experiments.xlsx', index=False)

# Register Libraries

In [ ]:
y4q1 = gcat.get_file(spreadsheet_name, fmt='pandas_excel')
libraries = y4q1.parse('Libraries', header=0)

created = server.post_sheet('/libraries/', libraries, verbose=True, dry_run=True)

In [ ]:
len(created)

In [ ]:
if created:
    libraries.to_excel('/tmp/libraries.xlsx', index=False)

# Register Replicates

In [ ]:
y4q1 = gcat.get_file(spreadsheet_name, fmt='pandas_excel')
replicates = y4q1.parse('Replicates', header=0)

created = server.post_sheet('/replicates/', replicates, verbose=True, dry_run=True)

In [ ]:
len(created)

In [ ]:
if created:
    replicates.to_excel('/tmp/replicates.xlsx', index=False)